# Кросс-валидация 


Я подробно рассмотрел различные методы кросс-валидации, а также написал общую идею кросс-валидации. Хорошую документацию по этой теме можно почитать вот здесь: https://scikit-learn.org/stable/modules/cross_validation.html

## Imports

In [325]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Пример данных

In [326]:
df = pd.read_csv("../DATA/Advertising.csv")

In [327]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


----
----
----
## Разбиваем данные на обучающий и тестовый наборы данных - Train | Test Split

0. Очищаем и масштабируем данные X и y (при необходимости)
1. Разбиваем данные на обучающий и тестовый наборы данных - как для X, так и для y
2. Обучаем объект Scaler на обучающих данных X
3. Применяем масштабирование (scale) для тестовых данных X
4. Создаём модель
5. Обучаем модель на обучающих данных X
6. Оцениваем модель на тестовых данных X (создавая предсказания и сравнивая их с Y_test)
7. Уточняем параметры модели, повторяя шаги 5 и 6

In [328]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# Масштабирование данных - SCALE DATA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

**Создаём модель**

In [329]:
from sklearn.linear_model import Ridge

In [330]:
# Мы осознанно указываем неудачное значение Alpha!
model = Ridge(alpha=100)

In [331]:
model.fit(X_train,y_train)

Ridge(alpha=100)

In [332]:
y_pred = model.predict(X_test)

**Оценка модели (evaluation)**

In [333]:
from sklearn.metrics import mean_squared_error

In [334]:
mean_squared_error(y_test,y_pred)

7.341775789034126

**Уточняем параметры и заново оцениваем модель**

In [335]:
model = Ridge(alpha=1)

In [336]:
model.fit(X_train,y_train)

Ridge(alpha=1)

In [337]:
y_pred = model.predict(X_test)

**Повторная оценка**

In [338]:
mean_squared_error(y_test,y_pred)

2.3190215794287514

Намного лучше! Мы можем повторять эти шаги до тех пор, пока не получим хорошие метрики производительности. (Ранее мы видели, как для этой цели использовать RidgeCV, но здесь мы можем обобщить процесс кросс-валидации для случая любой модели).

----
----
----
## Обучение - валидация - тестирование (Train | Validation | Test Split Procedure)

Этот подход ещё называют методом с задержкой ("hold-out"), потому что здесь мы не настраиваем параметры модели на финальном тестовом наборе данных, а используем тестовые данные *только* для оценки финального результата.

0. Очищаем и масштабируем данные X и y (при необходимости)
1. Разбиваем данные на обучающий, оценочный и тестовый наборы данных - как для X, так и для y
2. Обучаем объект Scaler на обучающих данных X
3. Масштабируем (scale) оценочные данные X
4. Создаём модель
5. Обучаем модель на обучающих данных X
6. Оцениваем модель на оценочных данных X (создавая предсказания и сравнивая их с Y_eval)
7. Уточняем параметры модели, повторяя шаги 5 и 6
8. Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)

In [339]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

In [340]:
###############################################################################################
#### Вызываем SPLIT дважды! Здесь мы создаём три набора данных - TRAIN, VALIDATION и TEST  #########
###############################################################################################
from sklearn.model_selection import train_test_split

# 70% данных определяем в обучающий набор, остальные 30% откладываем в сторону
X_train, X_OTHER, y_train, y_OTHER = train_test_split(X, y, test_size=0.3, random_state=101)

# Оставшиеся 30% разбиваем на оценочный и тестовый наборы данных
# Каждый будет по 15% от исходного набора данных 
X_eval, X_test, y_eval, y_test = train_test_split(X_OTHER, y_OTHER, test_size=0.5, random_state=101)

In [341]:
# Масштабируем данные (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_eval = scaler.transform(X_eval)
X_test = scaler.transform(X_test)

**Создаём модель**

In [342]:
from sklearn.linear_model import Ridge

In [343]:
# Мы осознанно указываем неудачное значение Alpha!
model = Ridge(alpha=100)

In [344]:
model.fit(X_train,y_train)

Ridge(alpha=100)

In [345]:
y_eval_pred = model.predict(X_eval)

**Оценка модели (Evaluation)**

In [346]:
from sklearn.metrics import mean_squared_error

In [347]:
mean_squared_error(y_eval,y_eval_pred)

7.320101458823866

**Уточняем параметры и повторно оцениваем модель**

In [348]:
model = Ridge(alpha=1)

In [349]:
model.fit(X_train,y_train)

Ridge(alpha=1)

In [350]:
y_eval_pred = model.predict(X_eval)

**Ещё раз оцениваем модель**

In [351]:
mean_squared_error(y_eval,y_eval_pred)

2.3837830750569853

**Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)**

In [352]:
y_final_test_pred = model.predict(X_test)

In [353]:
mean_squared_error(y_test,y_final_test_pred)

2.254260083800517

----
----
----
## Кросс-валидация с помощью cross_val_score

----

<img src="grid_search_cross_validation.png">

----

In [354]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

# Масштабирование данных (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [355]:
model = Ridge(alpha=100)

In [356]:
from sklearn.model_selection import cross_val_score

In [357]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [358]:
scores

array([-8.13059258, -7.95667991, -5.10132437, -7.43604224, -7.34397196])

In [359]:
# Среднее значение MSE scores (делаем это значение положительным)
abs(scores.mean())

7.193722211988183

**Уточняем модель на основе метрик**

In [360]:
model = Ridge(alpha=1)

In [361]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_val_score(model,X_train,y_train,
                         scoring='neg_mean_squared_error',cv=5)

In [362]:
# Среднее значение MSE scores (делаем это значение положительным)
abs(scores.mean())

3.1340960121990995

**Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)**

In [363]:
# Сначала нужно обучить модель!
model.fit(X_train,y_train)

Ridge(alpha=1)

In [364]:
y_final_test_pred = model.predict(X_test)

In [365]:
mean_squared_error(y_test,y_final_test_pred)

1.7929276024696261

----
----
----

# Кросс-валидация с помощью cross_validate

Функция cross_validate отличается от cross_val_score двумя аспектами:

эта функция позволяет использовать для оценки несколько метрик;

она возвращает не только оценку на тестовом наборе (test score), но и словарь с замерами времени обучения и скоринга, а также - опционально - оценки на обучающем наборе и объекты estimator.

В случае одной метрики для оценки, когда параметр scoring является строкой string, вызываемым объектом callable или значением None, ключи словаря будут следующими:
        
        - ['test_score', 'fit_time', 'score_time']

А в случае нескольких метрик для оценки, возвращаемый словарь будет содержать следующие ключи:

    ['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']

return_train_score по умолчанию принимает значение False, чтобы сэкономить вычислительные ресурсы. Чтобы посчитать оценки на обучающем наборе, достаточно установить этот параметр в значение True.

In [366]:
## Создаём X и y
X = df.drop('sales',axis=1)
y = df['sales']

# Делаем разбиение на TRAIN и TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

# Масштабируем данные (SCALE)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [367]:
model = Ridge(alpha=100)

In [368]:
from sklearn.model_selection import cross_validate

In [369]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=5)

In [370]:
scores

{'fit_time': array([0.0004847 , 0.00034904, 0.00048614, 0.00038028, 0.00027919]),
 'score_time': array([0.00052524, 0.00043797, 0.00040722, 0.00021982, 0.00019979]),
 'test_neg_mean_absolute_error': array([-2.26799776, -2.05485022, -1.71019531, -1.8960299 , -2.10651371]),
 'test_neg_mean_squared_error': array([-8.13059258, -7.95667991, -5.10132437, -7.43604224, -7.34397196]),
 'test_max_error': array([ -6.33935793,  -6.47142053,  -5.42622357, -10.12982458,
         -6.89608044])}

In [371]:
pd.DataFrame(scores)

,fit_time,score_time,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_max_error
0,0.000485,0.000525,-2.267998,-8.130593,-6.339358
1,0.000349,0.000438,-2.054850,-7.956680,-6.471421
2,0.000486,0.000407,-1.710195,-5.101324,-5.426224
3,0.000380,0.000220,-1.896030,-7.436042,-10.129825
4,0.000279,0.000200,-2.106514,-7.343972,-6.896080


In [372]:
pd.DataFrame(scores).mean()

fit_time                        0.000396
score_time                      0.000358
test_neg_mean_absolute_error   -2.007117
test_neg_mean_squared_error    -7.193722
test_max_error                 -7.052581
dtype: float64

**Уточняем модель на основе метрик**

In [373]:
model = Ridge(alpha=2)

In [374]:
# Варианты оценки модели:
# https://scikit-learn.org/stable/modules/model_evaluation.html
scores = cross_validate(model,X_train,y_train,
                         scoring=['neg_mean_absolute_error','neg_mean_squared_error','max_error'],cv=10)

In [375]:
pd.DataFrame(scores).mean()

fit_time                        0.000305
score_time                      0.000272
test_neg_mean_absolute_error   -1.332523
test_neg_mean_squared_error    -3.117189
test_max_error                 -4.160589
dtype: float64

**Вычисляем финальные метрики на тестовом наборе данных (после этого уже нельзя возвращаться и делать уточнения!)**

In [376]:
# Сначала нужно обучить модель!
model.fit(X_train,y_train)

Ridge(alpha=2)

In [377]:
y_final_test_pred = model.predict(X_test)

In [378]:
mean_squared_error(y_test,y_final_test_pred)

1.7921967991485475

----
----